# Python Final Project: Human Resource Data Analysis
By: Phionna Teo

**This project aims to explore three fundamental questions about the Human Resource Data Set:**
1. Is there a relationship between pay and performance score (adjusted for position)?
2. What is the overall diversity profile of the organization?
3. What are the best recruiting sources for the company to invest in?

In [1]:
import tablib
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
import numpy as np
from sklearn.linear_model import LinearRegression

#Import the dataset and store in a tablib Dataset
datafile = open("HRDataset_v9.csv","r")
data = tablib.import_set(datafile.read())
datafile.close()

list_of_headers = data.headers

In [2]:
#Method that takes in a Header Title and returns a set of values in that column
def get_set_of_column(Header):
    return set(data[Header])

#Method that takes in a dictionary, and values to append to it, returns nothing
def append_dict(dictionary, position, value):
    if position not in dictionary.keys():
        dictionary[position] = [value]
    else:
        dictionary[position].append(value)
    return

In [3]:
#A Dictionary where key is position, value is a list of pay and performance score
position_pay_perf = {}

#Populate the dictionary above using a for loop
for position in get_set_of_column("Position"):
    for employee in data.dict:
        if employee["Position"] == position and employee["Employment Status"] == "Active":
            pay_perf = (float(employee["Pay Rate"]), employee["Perf_ScoreID"])
            append_dict(position_pay_perf,position,pay_perf)

# What is the average pay rate per position?

Knowing the pay rates by position, lets first look at which position is paid the most, on average.

In [4]:
#Create a new Dataset for the Average Pay Table
avg_pay = tablib.Dataset()
avg_pay.headers = ['Position', 'Average Pay Rate']

for position in position_pay_perf:
    sum_pay = 0
    counter = 0
    for person in position_pay_perf[position]:
        sum_pay += person[0]
        counter += 1
    avg = sum_pay / counter
    avg_pay.append([position, avg])

avg_pay_sorted = avg_pay.sort("Average Pay Rate", reverse = True)

#This creates a file where you can see the average pay rate by position
avg_pay_file = open("Average Pay Sorted by Position", "w")
avg_pay_file.write(avg_pay_sorted.csv)
avg_pay_file.close()

#This prints a sorted table of 'Position' and the 'Average Pay Rate'
print("%-30s %s" %('Position', 'Average Pay Rate'))
for i in avg_pay_sorted.dict:
    print("%-30s $%.1f" %(i['Position'],i['Average Pay Rate']))

Position                       Average Pay Rate
President & CEO                $80.0
CIO                            $65.0
IT Director                    $65.0
IT Manager - Support           $64.0
BI Director                    $63.5
IT Manager - Infra             $63.0
IT Manager - DB                $62.0
Director of Operations         $60.0
Director of Sales              $60.0
Area Sales Manager             $55.3
Sales Manager                  $55.0
Shared Services Manager        $55.0
Data Architect                 $55.0
Sr. Network Engineer           $54.1
Production Manager             $53.6
Software Engineer              $52.2
Senior BI Developer            $51.2
BI Developer                   $45.2
Network Engineer               $41.1
Database Administrator         $36.5
Sr. Accountant                 $35.0
IT Support                     $28.5
Software Engineering Manager   $27.0
Accountant I                   $26.8
Production Technician II       $25.1
Production Technician I    

# Does higher performance correspond to higher pay?

Now that we found the variability in pay rate according to positions, lets analyze the variability of pay rate within each position. Does higher performance correspond to a higher pay rate?

For this analysis, lets weed out those positions that only have 2 or less data points, otherwise the regression is meaningless.

Lets also weed out 'PerformanceID' data points that have scores of '9' and '0' because a score of 9 indicates 'too early to review' and 0 is a probational '90-day-meets'.


In [5]:
new_position_pay_perf = {}
regression_position_pay_perf = {}

for position in position_pay_perf:
    for p in position_pay_perf[position]:
        if int(p[1]) <= 5 and int(p[1]) != 0:
            append_dict(new_position_pay_perf,position,p)

for po in new_position_pay_perf:
    if len(new_position_pay_perf[po]) > 2:
        regression_position_pay_perf[po] = new_position_pay_perf[po]

In [6]:
#A method that returns the r-squared value of the regression, takes in a dictionary and the position
def get_r_squared(dictionary,position):
    list_of_x = []
    list_of_y = []
    for p in dictionary[position]:
        list_of_x.append(int(p[1]))
        list_of_y.append(p[0])
    x = np.array(list_of_x).reshape((-1,1))
    y = np.array(list_of_y)

    model = LinearRegression().fit(x, y)
    score = model.score(x,y)
    return score

#A method that does the visualization, takes in dictionary,position and r-squared score, returns nothing
def scatter_plot_graph(dictionary,position,score,title):
    list_of_x = []
    list_of_y = []
    for p in dictionary[position]:
        list_of_x.append(int(p[1]))
        list_of_y.append(p[0])
    x = np.array(list_of_x)
    y = np.array(list_of_y)
    graph_title = "Pay Rate vs %s for %s, R-Square= %.4f" %(title, position, score)
    p = figure(plot_width=500, plot_height=500,title= graph_title,x_axis_label=title,y_axis_label="Pay Rate")

    p.circle(x,y, size=20, color="navy", alpha=0.5)

    return p

In [7]:
#Perform regression and data visualization for each position
list_of_r2 = []
list_of_plots = []

for position in regression_position_pay_perf:
    r_sq = get_r_squared(regression_position_pay_perf, position)

    list_of_r2.append(r_sq)
    p = scatter_plot_graph(regression_position_pay_perf,position,r_sq,"Performance")
    list_of_plots.append(p)

output_notebook()
show(column(list_of_plots))

Loading BokehJS ...

In [8]:
print("The average R-squared value is %.2f." %(sum(list_of_r2)/len(list_of_r2)))

The average R-squared value is 0.08.


Based on the scatter plots and R-squared values of each position, it seems that high performance score rarely correlated with higher pay rate. The variability in the pay rate for each position is not explained by their performance.

The position with the highest R-squared is IT Support, but looking at the scatter plot, the employee who had a higher performance score in fact earned the lowest.

This could be a problem that the HR department could look into - to discuss if it is necessary to implement a compensation or bonus system that rewards higher performance.

Since performance doesnt explain the variability in pay rate for each position, then what does? Could it be seniority? Let's look at the data of Number of Days that that employee has worked for the company and the corresponding pay rate.

# Do employees who have worked longer receive a higher pay rate?

In [9]:
#A Dictionary where key is position, value is a list of pay and days employed
position_pay_seniority = {}

#Populate the dictionary above using a for loop
for position in get_set_of_column("Position"):
    for employee in data.dict:
        if employee["Position"] == position and employee["Employment Status"] == "Active":
            pay_seniority = (float(employee["Pay Rate"]), employee["Days Employed"])
            append_dict(position_pay_seniority,position,pay_seniority)
    
#Once again we weed out those positions with data points < 2
new_position_pay_seniority = {}

for po in position_pay_seniority:
    if len(position_pay_seniority[po]) > 2:
        new_position_pay_seniority[po] = position_pay_seniority[po]

In [10]:
#We perform the same regression and visualization analysis for days hired and pay rate
list_of_r2_seniority = []
list_of_plots_seniority = []

for position in new_position_pay_seniority:
    r_sq = get_r_squared(new_position_pay_seniority, position)

    list_of_r2_seniority.append(r_sq)
    p = scatter_plot_graph(new_position_pay_seniority,position,r_sq,"Days Hired")
    list_of_plots_seniority.append(p)

output_notebook()
show(column(list_of_plots_seniority))

Loading BokehJS ...

In [11]:
print("The average r-squared for this analysis is %.2f." %(sum(list_of_r2_seniority)/len(list_of_r2_seniority)))

The average r-squared for this analysis is 0.12.


We can interpret this as: on average, the number of days that you have worked at this company accounts for about 12% of the variability of your pay rate vis-a-vis peers who hold the same position as you. This is higher than the R-squared value for performance score, but overall it is still very low.

It is important for the company to be transparent to their staff about what accounts for the variability in pay rate, as it is a sensitive issue that can create a lot of conflict if not managed well.

# What is the overall diversity profile of the company?

Lets analyze the overall diversity profile of the company and visualize the data. Here, I use 3 diversity categories: Gender, Race Description and Age. <br />

First, lets analyze their gender ratio:

## Gender

In [12]:
#A method that takes in a string that is in the list of headers of the data, and counts the number of employees
# according to the categories within the column 'header'
#Returns a dictionary with key as each category, value as the count of employees that falls within that category

def count_by_category(Header):
    set_of_category = list(get_set_of_column(Header))
    count_of_category = {}

    for category in set_of_category:
        for employee in data.dict:
            if employee["Employment Status"] == 'Active':
                if employee[Header] == category:
                    append_dict(count_of_category, employee[Header],1)

    sum_of_categories = sum_items_in_dict(count_of_category)
    return sum_of_categories

#A method that takes in a string that is in the list of headers of the data, and a dictionary, the sum of categories
#It returns a dictionary with key as each category, and value as the % count of the total employees

def percentage_by_category(Header, sum_of_categories):
    list_of_values = []
    for category in sum_of_categories:
        list_of_values.append(sum_of_categories[category])
    percentage_of_values = get_percentage(list_of_values)
    percentage_of_category = {}
    i = 0
    for category in sum_of_categories:
        percentage_of_category[category] = percentage_of_values[i]
        i +=1
    return percentage_of_category

#A method that takes in a full list of values and returns a list of their % out of the values in the list

def get_percentage(list_of_values):
    sum_values = sum(list_of_values)
    new_list = []
    for i in list_of_values:
        percent = (i / sum_values) * 100
        new_list.append(percent)
    return new_list

#A method that takes in a dictionary, and returns a dictionary that is the sum of the length of items in the value
def sum_items_in_dict(dictionary):
    new_dict = {}
    for item in dictionary:
        new_dict[item] = len(dictionary[item])
    return new_dict

In [13]:
count_by_gender = count_by_category("GenderID")
female_counter = count_by_gender['0']
male_counter = count_by_gender['1']

#A dictionary with more user friendly name- Female and Male
count_by_gender_names = {'Females': female_counter, 'Males': male_counter}

In [14]:
from math import pi
import pandas as pd
from bokeh.transform import cumsum
from bokeh.palettes import Spectral6, Spectral5, Pastel1, Category20c

def piechart(x_value,color,title):
    x = x_value

    datap = pd.Series(x).reset_index(name='value').rename(columns={'index':'Race'})
    datap['angle'] = datap['value']/datap['value'].sum() * 2*pi
    datap['color'] = color

    p = figure(plot_height=500, plot_width = 900, title=title,
            tools="hover", tooltips="@Race: @value")

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend='Race', source=datap)
    p.legend.location = "top_right"
    show(p)

piechart(count_by_gender_names, ["pink","navy"],"Gender Diversity in the Organization")

percentage_by_gender = percentage_by_category("GenderID",count_by_gender_names)
Female_percentage = percentage_by_gender['Females']
Male_percentage = percentage_by_gender['Males']

print("The percentage of females in the company is %.2f%% and the percentage of males in the company is %.2f%%." %(Female_percentage, Male_percentage))

The percentage of females in the company is 55.19% and the percentage of males in the company is 44.81%.


The organization seems to have a higher proportion of females in the company! But what positions do they undertake? Let's analyze the job titles of each respective gender. 

In [15]:
#A method that takes in a list of categories, a list of position titles, and the category in which we should count the positions
#Returns a list of categories, within each list, a list of position and the sum of people in that category who occupy that position

def job_count_by_category(set_of_categories,positions,header):
    list_of_dict = []
    for category in set_of_categories:
        jobs_category = {}
        for job in positions:
            for employee in data.dict:
                if employee["Employment Status"] == 'Active':
                    if employee[header] == category and employee["Position"] == job:
                        append_dict(jobs_category, employee["Position"],1)
        sum_jobs_category = sum_items_in_dict(jobs_category)
        list_of_dict.append([category, sum_jobs_category])

    return list_of_dict

positions = avg_pay_sorted['Position'] #This will get the list of positions already sorted by pay

#A method that takes in the list of positions and dictionary
#Returns a list of positions and a count of that position, with '0' for none
def position_counting(positions,dictionary):
    list_positions = []
    for position in positions:
        if position in dictionary.keys():
            list_positions.append(dictionary[position])
        else:
            list_positions.append(0)
    return list_positions
    

In [16]:
total_dict_sum = job_count_by_category(["Female","Male"],positions,"Sex")
female_jobs_dict_sum = total_dict_sum[0][1]
male_jobs_dict_sum = total_dict_sum[1][1]

fem_positions = position_counting(positions,female_jobs_dict_sum)
male_positions = position_counting(positions, male_jobs_dict_sum)

all_positions_count=[]
male = 0
for fem in fem_positions:
    all_positions_count.append(fem + male_positions[male])
    male += 1

In [17]:
#A method that takes in a dictionary that has the sum by category, and returns the dictionary as percentage by category
def job_percentage_by_category(sum_by_category_dict):
    list_of_dict_counted= []
    for i in range(len(sum_by_category_dict)):
        list_of_dict_counted.append([sum_by_category_dict[i][0], position_counting(positions,sum_by_category_dict[i][1])])

    new_list_of_dict_counted = {}

    for cat in list_of_dict_counted:
        i = 0
        new_list = []
        while i < len(all_positions_count):
            new_list.append(cat[1][i] / all_positions_count[i] * 100)
            i += 1
        new_list_of_dict_counted[cat[0]] = new_list
    return new_list_of_dict_counted

In [18]:
job_percentage_by_gender = job_percentage_by_category(total_dict_sum)

In [19]:
from bokeh.models import ColumnDataSource

colors = ["pink", "navy"]
all_positions = positions
gender = ["Female", "Male"]

exports = {'all_positions' : all_positions,
           'Female'   : job_percentage_by_gender['Female'],
           'Male'   : job_percentage_by_gender['Male']}

p = figure(y_range=all_positions, plot_height=700, plot_width=800, x_range=(0, 100), title="Job Representation By Gender (%) - Sorted from Lowest Pay to Highest",
           toolbar_location=None,x_axis_label="% of Gender in Each Position")

p.hbar_stack(gender, y='all_positions', color = colors, height=0.9, source=ColumnDataSource(exports))

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

Some insights we can obtain from this visualization: the highest-paid executives are females, but the lowest-paid jobs are also occupied by females. It may be hard to draw an immediate conclusion, but the visualization does not point to a severe lack of representation in the highest-paying and lowest-paying segments of the company. </br>

Finally, lets look at the average pay rate of females and males to see if there is a pay gap. 



In [20]:
#A method that takes in a list of categories and the header of that category
#Returns a dictionary of the pay gap between the categories
def pay_gap(category_set, category_header):
    new_dict_pay_rate = {}
    for category in category_set:
        sum_pay_rate = 0
        counter_pay = 0
        for employee in data.dict:
            if employee["Employment Status"] == 'Active':
                if employee[category_header] == category:
                    sum_pay_rate += float(employee["Pay Rate"])
                    counter_pay += 1
        new_dict_pay_rate[category] = sum_pay_rate / counter_pay

    return new_dict_pay_rate

avg_fem_pay = pay_gap(['Female','Male'],"Sex")["Female"]
avg_male_pay = pay_gap(['Female','Male'],"Sex")["Male"]

print("The average pay rate for females in this organization is $%.2f and for males, $%.2f." %(avg_fem_pay, avg_male_pay))
print("Despite the fact that the highest-paid executive in the company is a female, there is still an average $%.2f pay gap, based on the data." %(avg_male_pay - avg_fem_pay))



The average pay rate for females in this organization is $32.15 and for males, $35.91.
Despite the fact that the highest-paid executive in the company is a female, there is still an average $3.76 pay gap, based on the data.


## Ethnicity

In [21]:
count_of_race_desc = count_by_category("RaceDesc")
percentage_of_race_desc = percentage_by_category("RaceDesc", count_of_race_desc)

for race in percentage_of_race_desc:
    print("The percentage of employees who identify as %s is %.2f%%." %(race, percentage_of_race_desc[race]))

The percentage of employees who identify as Hispanic is 1.09%.
The percentage of employees who identify as White is 62.84%.
The percentage of employees who identify as Asian is 9.84%.
The percentage of employees who identify as American Indian or Alaska Native is 1.64%.
The percentage of employees who identify as Two or more races is 4.92%.
The percentage of employees who identify as Black or African American is 19.67%.


In [22]:
piechart(count_of_race_desc,Spectral6,"Ethnic Diversity in the Organization")

Based on this visualization, it appears that the organization is not at all diverse. What titles do these respective employees hold?

In [23]:
job_count_by_race = job_count_by_category(get_set_of_column("RaceDesc"),positions,"RaceDesc")
percentage_job_by_race = job_percentage_by_category(job_count_by_race)
races = [race for race in percentage_job_by_race]

In [24]:
from bokeh.core.properties import value, field

colors = Spectral6
all_positions = positions
race = races

exports = {'all_positions' : all_positions,
           'Two or more races'   : percentage_job_by_race["Two or more races"],
           'Asian'   : percentage_job_by_race["Asian"],
           'American Indian or Alaska Native': percentage_job_by_race["American Indian or Alaska Native"],
           'Hispanic': percentage_job_by_race["Hispanic"],
           'Black or African American': percentage_job_by_race["Black or African American"],
           'White': percentage_job_by_race["White"]}

p = figure(y_range=all_positions, plot_height=700, plot_width=900, x_range=(0, 100), title="Job Representation By Ethnicity (%) - Sorted from Lowest Pay to Highest",x_axis_label="% of Ethnicity in Each Position")

p.hbar_stack(race, y='all_positions', color = colors, height=0.9, source=ColumnDataSource(exports), legend=[value(x) for x in race])

p.legend.location = "bottom_right"
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

The positions of 'Area Sales Manager' and 'Production Technicians' seem to have a lot more diversity than other positions. Another trend observed is that from the director level and above, there is a significant lack of ethnic diversity. This could be an actionable insight for the HR department to pursue. </br>

Finally, lets look at the average pay rate for each ethnic category, unadjusted for position. 

In [25]:
set_of_race_desc = get_set_of_column('RaceDesc')
avg_pay_rate_race = pay_gap(set_of_race_desc,"RaceDesc")

races = []
pay_rates = []
for race in avg_pay_rate_race:
    print("The average pay rate for the ethnic category of '%s' is $%.2f." %(race, avg_pay_rate_race[race]))
    races.append(race)
    pay_rates.append(avg_pay_rate_race[race])


The average pay rate for the ethnic category of 'Hispanic' is $58.00.
The average pay rate for the ethnic category of 'White' is $32.87.
The average pay rate for the ethnic category of 'Asian' is $32.99.
The average pay rate for the ethnic category of 'American Indian or Alaska Native' is $31.50.
The average pay rate for the ethnic category of 'Two or more races' is $35.36.
The average pay rate for the ethnic category of 'Black or African American' is $35.83.


In [26]:
def barchart(x_cat,y_cat,colors,title,y_axis_title):
    race = x_cat
    counts = y_cat

    # sorting the bars means sorting the range factors
    sorted_race = sorted(race, key=lambda x: counts[race.index(x)])

    p = figure(x_range=sorted_race, plot_height=350, plot_width = 900, title=title,
               toolbar_location=None, tools="",y_axis_label=y_axis_title)

    p.vbar(x=race, top=counts, width=0.7,color= colors)

    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    show(p)
    
barchart(races,pay_rates, Spectral6, "Avg Pay Rate by Ethnic Category, Sorted","Average Pay Rate")

From this visualization, it seems that on average, in this organization, employees identifying as 'Hispanic' have the highest average pay rate, but we must bear in mind that the sample size is 2 for this category, hence the result of the average pay rate can be very skewed. 

## Age
Finally lets take a look at the diversity of the employees in terms of their ages.

In [27]:
#Lets first create a new column in our dataset of our age categories: 20-30, 31-40, 41-50, 51-60, 61-70
if 'Age Category' not in list_of_headers:
    value = ""
    age_categories = []
    for employee in data.dict:
        age_int = int(employee['Age'])
        if age_int <= 30:
            value = "20-30"
        elif age_int <=40:
            value = "31-40"
        elif age_int <=50:
            value = "41-50"
        elif age_int <=60:
            value = "51-60"
        else:
            value = "61-70"
        age_categories.append(value)
    data.append_col(age_categories, 'Age Category')


In [28]:
count_of_age = count_by_category("Age Category")
percentage_by_age = percentage_by_category("Age Category", count_of_age)

for age in percentage_by_age:
    print("The percentage of employees that fall within the %s age range is %.2f%%" %(age, percentage_by_age[age]))

The percentage of employees that fall within the 31-40 age range is 48.09%
The percentage of employees that fall within the 20-30 age range is 19.67%
The percentage of employees that fall within the 41-50 age range is 22.40%
The percentage of employees that fall within the 51-60 age range is 8.20%
The percentage of employees that fall within the 61-70 age range is 1.64%


In [29]:
piechart(count_of_age,Pastel1[5],"Diversity by Age in the Organization")

Majority of employees fall within the age range of 31-40, and those who are 50 and above only form approx 9% of the workforce in this company. Once again, we can analyze the job distribution by age, as well as the average pay rate by age.

In [30]:
job_count_by_age = job_count_by_category(get_set_of_column('Age Category'),positions,'Age Category')
job_percentage_by_age = job_percentage_by_category(job_count_by_age)

age_categories = ["20-30","31-40","41-50","51-60","61-70"]

In [31]:
from bokeh.core.properties import value
colors = Pastel1[5]
all_positions = positions
ages = ['20-30','31-40','41-50','51-60','61-70']

exports = {'all_positions' : all_positions,
           '20-30'   : job_percentage_by_age["20-30"],
           '31-40'   : job_percentage_by_age["31-40"],
           '41-50': job_percentage_by_age["41-50"],
           '51-60': job_percentage_by_age["51-60"],
           '61-70': job_percentage_by_age["61-70"]}

p = figure(y_range=all_positions, plot_height=700, plot_width=900, x_range=(0, 100), title="Job Representation By Age (%) - Sorted from Lowest Pay to Highest", x_axis_label="% of Age in Each Position")

ages1 = ['20-30','31-40','41-50','51-60','61-70']

p.hbar_stack(ages, y='all_positions', color = colors, height=0.9, source=ColumnDataSource(exports),legend = [value(x) for x in ages])

p.legend.location = "bottom_right"
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

Some insights: </br>

-The youngest employees (aged 20-30) are generally still working in the lower-paying jobs. </br>

-Once again, it seems like the positions of "Production Technician I" and "Area Sales Manager" has greater age diversity!

-The President & CEO, which is the highest-paying position, is older than most people in the company. 

In [32]:
avg_pay_rate_age = pay_gap(age_categories,"Age Category")

ages = []
ages_pay_rate = []
for age in avg_pay_rate_age:
    ages.append(age)
    ages_pay_rate.append(avg_pay_rate_age[age])

barchart(ages,ages_pay_rate, Pastel1[5], "Average Pay Rate by Age Categories, Sorted","Average Pay Rate")

# Which recruiting sources has been providing the company with the best-performing employees?

According to the dataset, the company has used 23 different recruiting sources to staff its company over the years. But are all the sources providing high-quality employees? Which source should the company invest into more given the different costs of each source?

In [33]:
sources = get_set_of_column("Employee Source")
no_of_sources = len(sources)

In [34]:
#A method that was similar to count_by_category, but takes into account all employees, rather than just the active ones
def count_by_category_all(Header):
    set_of_category = list(get_set_of_column(Header))
    count_of_category = {}

    for category in set_of_category:
        for employee in data.dict:
            if employee[Header] == category:
                append_dict(count_of_category, employee[Header],1)

    sum_of_categories = sum_items_in_dict(count_of_category)
    return sum_of_categories

In [35]:
count_by_source = count_by_category_all("Employee Source")
percentage_by_source= percentage_by_category("Employee Source", count_by_source)

sources = []
number = []

#Display only the top 8 sources (by no. of employees)
for source in count_by_source:
    if count_by_source[source] > 16:
        sources.append(source)
        number.append(count_by_source[source])

In [36]:
from bokeh.palettes import Set3 

barchart(sources,number,Set3[8], "Top 8 Recruiting Sources, Sorted","Number of Employees")

In [37]:
percentage_by_source_sorted = {}
print("%-50s %s" %("Recruiting Sources","% of Employees Recruited by this Channel"))
for key, value in sorted(percentage_by_source.items(), key=lambda item: item[1],reverse=True):
    print("%-50s %.2f%%" %(key, value))

Recruiting Sources                                 % of Employees Recruited by this Channel
Employee Referral                                  10.00%
Diversity Job Fair                                 9.35%
Search Engine - Google Bing Yahoo                  8.06%
Monster.com                                        7.74%
Pay Per Click - Google                             6.77%
Professional Society                               6.45%
Newspager/Magazine                                 5.81%
MBTA ads                                           5.48%
Billboard                                          5.16%
Vendor Referral                                    4.84%
Glassdoor                                          4.52%
Word of Mouth                                      4.19%
Website Banner Ads                                 4.19%
On-campus Recruiting                               3.87%
Social Networks - Facebook Twitter etc             3.55%
Other                                              2

Employee referral has provided the most number of employees, followed by Diversity Job Fair and Search Engine- Google Bing Yahoo. 

## Employee Performance 
How are these employees doing in terms of performance?

In [38]:
employment_source_perf = {}

for source in sources:
    for employee in data.dict:
        score = int(employee["Perf_ScoreID"])
        if employee["Employee Source"] == source and score <= 5 and score != 0: #Once again, lets remove performance scores of 0 and 9
            append_dict(employment_source_perf,source,score)

In [39]:
#Get Average Performance Score by Source
avg_perf_by_source = {}
for source in employment_source_perf:
    avg_perf_by_source[source] = sum(employment_source_perf[source])/len(employment_source_perf[source])

sources = []
scores = []
for source in avg_perf_by_source:
    sources.append(source)
    scores.append(avg_perf_by_source[source])


In [40]:
barchart(sources,scores,Set3[8],"Average Performance Score by Recruiting Source (Top 8 Feeders), Sorted","Average Performance Score")

It seems like on average, employee referral has provided the best-performing employees, followed by Professional Society.

## Recruiting Costs
How much has the company spent on these recruiting sources? Are the high-quality employees obtained through these channels worth the cost? 

In [41]:
datafile2 = open("recruiting_costs.csv","r")
recruiting = tablib.import_set(datafile2.read())
datafile.close()

total_recruiting_costs = {}
for source in recruiting.dict:
    total_recruiting_costs[source['Employment Source']] = int(source['Total'])

print("%-50s %s" %("Recruiting Source", "Total Cost"))
for key, value in sorted(total_recruiting_costs.items(), key=lambda item: item[1],reverse=True):
    print("%-50s $%d" %(key, value))

Recruiting Source                                  Total Cost
MBTA ads                                           $10980
Diversity Job Fair                                 $10021
Newspager/Magazine                                 $8291
Careerbuilder                                      $7790
On-campus Recruiting                               $7500
Website Banner Ads                                 $7143
Billboard                                          $6192
Monster.com                                        $5760
Social Networks - Facebook Twitter etc             $5573
Search Engine - Google Bing Yahoo                  $5183
Other                                              $3995
Pay Per Click - Google                             $3509
Pay Per Click                                      $1323
Professional Society                               $1200
Company Intranet - Partner                         $0
Employee Referral                                  $0
Glassdoor                     

In [42]:
#Lets look at the cost of recruiting source per employee

recruiting_cost_per_employee = {}

for source in total_recruiting_costs:
    recruiting_cost_per_employee[source] = total_recruiting_costs[source] / count_by_source[source]

print("%-50s %s" %("Recruiting Source", "Average Cost / Employee Gained"))
for key, value in sorted(recruiting_cost_per_employee.items(), key=lambda item: item[1],reverse=True):
    print("%-50s $%d" %(key, value))

Recruiting Source                                  Average Cost / Employee Gained
Careerbuilder                                      $7790
Pay Per Click                                      $1323
MBTA ads                                           $645
On-campus Recruiting                               $625
Website Banner Ads                                 $549
Social Networks - Facebook Twitter etc             $506
Newspager/Magazine                                 $460
Other                                              $443
Billboard                                          $387
Diversity Job Fair                                 $345
Monster.com                                        $240
Search Engine - Google Bing Yahoo                  $207
Pay Per Click - Google                             $167
Professional Society                               $60
Company Intranet - Partner                         $0
Employee Referral                                  $0
Glassdoor                

Now let's look at the cost of each recruiting source vis-a-vis the quality of employees recruited from that channel.

In [43]:
new_dict = {}
for perf in avg_perf_by_source:
    new_dict[perf] = [avg_perf_by_source[perf], recruiting_cost_per_employee[perf]]

cost_list = []
perf_list = []
names_list = []
for new in new_dict:
    names_list.append(new)
    perf_list.append(new_dict[new][0])
    cost_list.append(new_dict[new][1])

In [44]:
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, LabelSet, Label
from bokeh.palettes import plasma
from bokeh.transform import transform

list_x = perf_list
list_y = cost_list
recruiting_source = names_list

source = ColumnDataSource(data=dict(x=list_x, y=list_y, desc=recruiting_source))
hover = HoverTool(tooltips=[
    ('source', '@desc'),
    ("(Average Performance Score,Average Cost)", "(@x, @y)"),
])
mapper = LinearColorMapper(palette=plasma(256), low=min(list_y), high=max(list_y))

p = figure(plot_width=900, plot_height=400, y_range=(-50,750),x_range=(2.8,3.5), tools=[hover], title="Cost vs Performance Score for each Recruiting Source")
p.circle('x', 'y', size=20, source=source, fill_color=transform('y', mapper))

labels = LabelSet(x='x', y='y', text='desc', level='glyph', x_offset=-30, y_offset=12, source=source, render_mode='canvas')

p.add_layout(labels)
p.xaxis[0].axis_label = 'Average Performance Score for Employee'
p.yaxis[0].axis_label = 'Average cost/employee'
output_notebook()
show(p)

Loading BokehJS ...

Based on this visualization, it seems that **Employee Referral** has been the best recruiting source for the company. It has the lowest cost, and produces the highest-performing employees. The second best source by these metrics is **Professional Society**. The HR Department can act upon this insight by investing more into their employee referral program, and continue building connections with professional societies.

Dataset From: https://www.kaggle.com/rhuebner/human-resources-data-set